# Import Dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df = pd.read_csv("gojek.csv", low_memory=False)
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df['userName'].unique()

In [ ]:
df['score'].unique()

In [ ]:
df.duplicated().sum()

# Data Cleaning

In [ ]:
df = df[['content', 'score', 'thumbsUpCount', 'at']]
df['at'] = pd.to_datetime(df['at'])
df['month'] = df['at'].dt.to_period('M')
df.head()

In [ ]:
df1 = df[:20000]

In [ ]:
# Inisialisasi stemmer dan stopword remover dari Sastrawi
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

stopword_factory = StopWordRemoverFactory()
stopword_remover = stopword_factory.create_stop_word_remover()

# Daftar stopwords kustom
custom_stopwords = ['nya', 'aplikasi', 'yg', 'yang', 'gak', 'driver']

# Fungsi untuk stemming dan stopword removal
def process_text(text):
    # Hapus stopwords bawaan Sastrawi
    text_no_stopwords = stopword_remover.remove(text)
    
    # Hapus stopwords kustom
    words = text_no_stopwords.split()
    filtered_words = [word for word in words if word.lower() not in custom_stopwords]
    text_no_custom_stopwords = ' '.join(filtered_words)
    
    # Lakukan stemming
    stemmed_text = stemmer.stem(text_no_custom_stopwords)
    return stemmed_text

# Terapkan proses pada kolom 'content'
df1['content_processed'] = df1['content'].apply(process_text)

# Tampilkan hasil
print(df)

In [ ]:
df1

# EDA

In [ ]:
fig = px.pie(df, names='score', title='Distribusi Rating Gojek', color='score',
             color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_traces(textinfo='percent+label')
fig.show()

In [ ]:
# Comment dengan thumbsUpCount terbanyak
df['content'].iloc[df['thumbsUpCount'].idxmax()]

In [ ]:
# jumlah comment per bulan
monthly_comment = df['month'].groupby(df['month']).count()
monthly_comment



In [ ]:
#line chart jumlah comment per bulan
plt.figure(figsize=(12, 6))
monthly_comment.plot(kind='line')
plt.title('Jumlah Komentar per Bulan')
plt.xlabel('Bulan')
plt.ylabel('Jumlah Komentar')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

Puncak tertinggi terjadi sekitar 2019 sampai 2020, dengan jumlah komentar mencapai hampir 60,000 per bulan. Ini bisa mencerminkan periode aktivitas pengguna yang sangat tinggi, mungkin karena promosi besar, peningkatan penggunaan layanan selama pandemi COVID-19 (misalnya, lebih banyak orang menggunakan layanan pengiriman), atau peluncuran fitur baru yang memicu banyak diskusi.

# Sentimen Analysis

In [ ]:
# Distribusi score
score_distribution = df1.groupby('score').size()
print("Distribusi Score:\n", score_distribution)

# Kata-kata yang sering muncul untuk setiap score
from collections import Counter
for score in df1['score'].unique():
    words = ' '.join(df1[df1['score'] == score]['content_processed']).split()
    word_freq = Counter(words).most_common(10)
    print(f"\nKata yang sering muncul untuk score {score}:")
    print(word_freq)

## Insight:
Kepuasan Pengguna (Skor 5 dan 4)
- Mayoritas ulasan masih memberikan skor 5, menunjukkan
tingkat kepuasan pengguna yang sangat tinggi secara keseluruhan.
- Skor 4 juga menunjukkan kepuasan dengan sedikit kritik ringan, mirip dengan dataset sebelumnya.
- Kata "cepat" dan "mudah" menunjukkan bahwa kecepatan dan kemudahan layanan menjadi nilai tambah yang signifikan.
- rekomendasi:
Pertahankan kecepatan dan kemudahan layanan, karena ini menjadi alasan utama kepuasan.
Promosikan fitur-fitur yang dianggap "membantu" untuk mempertahankan loyalitas pengguna.

Ketidakpuasan Pengguna (Skor 1 dan 2)
- Skor 1 menunjukkan keluhan utama terkait keterlambatan ("lama"), proses pemesanan ("pesan").
- Skor 2 juga mengeluhkan keterlambatan ("lama"), masalah pemesanan ("pesan" 8 kali).
- Rekomendasi:
Prioritaskan peningkatan kecepatan layanan, terutama untuk pengiriman atau respons (misalnya, optimasi rute driver atau sistem antrian).
Perbaiki sistem pemesanan (GoFood, GoRide, dll.) untuk mengurangi masalah seperti keterlambatan atau pesanan yang tidak terproses ("pesan").

Pengguna Netral (Skor 3)
- Skor 3 menunjukkan campuran pujian dan kritik, dengan fokus pada keterlambatan ("lama") dan pengalaman pemesanan makanan ("makan").
- Tingkatkan layanan GoFood dengan mempercepat pengiriman makanan, karena "makan" dan "lama" sering muncul.

In [ ]:
for score in range(5, 0, -1):
    # Ambil teks dari 'content_processed' untuk skor tertentu
    text = ' '.join(df1[df1['score'] == score]['content_processed'].dropna())
    
    # Buat word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white',
                         min_font_size=10).generate(text)
    
    # Tampilkan word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Word Cloud untuk Skor {score}')
    plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import joblib

#Buat label sentimen
def label_sentiment(score):
    if score >= 4:
        return 'positif'
    elif score == 3:
        return 'netral'
    else:
        return 'negatif'

df1['sentiment'] = df1['score'].apply(label_sentiment)

#Ekstraksi fitur dengan TF-IDF
vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1, 2))  # Unigram dan bigram
X = vectorizer.fit_transform(df1['content_processed'])
y = df1['sentiment']

# Seimbangkan data dengan SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Optimasi model SVM dengan Grid Search
param_grid = {
    'C': [0.1, 1, 10],  # Parameter regularisasi
    'kernel': ['linear', 'rbf'],  # Jenis kernel
    'class_weight': ['balanced', None]  # Penanganan data tidak seimbang
}

model = SVC(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Tampilkan parameter terbaik
print("Parameter Terbaik:", grid_search.best_params_)
print("Skor Terbaik (F1 Macro):", grid_search.best_score_)

# Evaluasi model terbaik pada data uji
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("\nLaporan Klasifikasi untuk Model Teroptimasi:")
print(classification_report(y_test, y_pred))

# Simpan model dan vectorizer
joblib.dump(best_model, 'optimized_sentiment_model.pkl')
joblib.dump(vectorizer, 'optimized_tfidf_vectorizer.pkl')

# Uji prediksi pada ulasan baru
def predict_sentiment(new_review):
    new_review_tfidf = vectorizer.transform([new_review])
    prediction = best_model.predict(new_review_tfidf)[0]
    return prediction


In [ ]:
new_reviews = ["layanan sangat lambat", "gojek sangat membantu", "cukup baik tapi kadang lama"]
for review in new_reviews:
    sentiment = predict_sentiment(review)
    print(f"Ulasan: '{review}' -> Sentimen: {sentiment}")